# Cross-shore transport

## First import some necessary packages

In [1]:
import logging
import pathlib
import sys
import warnings

import colorcet as cc
import dotenv
import geopandas as gpd
import holoviews as hv
import hvplot.pandas  # noqa: API import
import numpy as np
import pandas as pd
import panel as pn
import pooch

from bokeh.models import PanTool, WheelZoomTool, HoverTool
from bokeh.resources import INLINE
import bokeh.io

import coastal_dynamics as cd

# Activate Panel extension to make interactive visualizations
pn.extension()

In [2]:
# # Read questions from cloud storage
questions = cd.read_questions(
    "./6_cross_shore_transport.json"
    # "az://coastal-dynamics/questions/5_morphodynamics_upper_shoreface.json",
    # storage_options={"account_name": "coclico"},
)

# (Cross-shore) sediment transport
Welcome to the notebook of week 6! The main topic of this notebook is cross-shore sediment transport (chapter 7 of the book, with some extra attention for section 7.5 specifically). We will start with a small introduction, followed by a more detailed look into figure 7.21 from the book.

## Introduction
Remember from chapter 5 that the velocity $u$ close to the bed can be assumed to consists of a wave group-averaged component $\bar{u}$, a short-wave-averaged oscillatory component $u_{lo}$ and a short-wave component $u_{hi}$:
$$
u = \bar{u} + u_{lo} + u_{hi}
$$
We are interested in net sediment transport. Here we use the third and fourth odd velocity moment as a proxy for this (which is a gross simplification, but a workable one). If $\bar{u} \ll u_{lo} \ll u_{hi}$, the three most important terms in the third odd velocity moment are:
$$
\left\langle u \left|u\right|^2\right\rangle = 3 \left\langle \bar{u} \left|u_{hi}\right|^2\right\rangle + \left\langle u_{hi} \left|u_{hi}\right|^2\right\rangle + 3 \left\langle u_{lo} \left|u_{hi}\right|^2\right\rangle + ...
$$
Each of these terms is fully explained in the book (so you should definitly read up on this), but in short, each of these terms refers to the following process:
* $3 \left\langle \bar{u} \left|u_{hi}\right|^2\right\rangle$ : sediment stirred up by short waves and transported by the mean current
* $\left\langle u_{hi} \left|u_{hi}\right|^2\right\rangle$ :  sediment stirred up by short waves and transported by the high-frequency waves
* $3 \left\langle u_{lo} \left|u_{hi}\right|^2\right\rangle$ : sediment stirred up by short waves and transported by the low-frequency waves 

Each of the velocity moments can be directly measured in a flume, which is what Roelvink and Stive (1989) did. In quite some quasi-steady sediment transport formulations, one can recognize the velocity moments. For instance, have a look at Eqs. 6.48a and 6.48b in the textbook. 

It is possible to derive an equilibrium profile mathematically by balancing onshore and offshore terms. Bowen (1980) does this analytically for the middle and lower shoreface by balancing onshore transport by short waves and offshore transport by gravity.

For the present exercise, we are interested in Figure 7.21 and how it changes for different wave conditions and bathymetry. However, we are limited by the lack of direct observations. [Tinker et al (2009)](https://doi.org/10.1016/j.csr.2009.03.011) present a solution. They performed a large amount of suspended transport measurements and fitted a shape function for transport due to both mean and oscillatory flow in the surf/shoaling zone, as well as onshore and offshore transport in the swash/inner surf zone. Though the paper is very interesting (and we definitely recommend checking it out), you are not required to know it for this exercise. You will hear more about it in the Coastal module as well, should you choose it!

However, before getting into the Tinker et al (2009) paper, let's first have a look at Figure 7.21 from the book. Use the velocity moments as a proxy for sediment transport. The total (net) transport consists of multiple gross contributions. For instance, the upper panel of figure 7.21 gives the total third odd moment as well its three components given by the right-hand-side of the above equation. 


![image](./6_fig720.png)

Using this figure, try to answer the questions below.

In [3]:
## @Kevin: I updated the json and the questions have changed, but the correct/incorrect answers do not work and also the feedback that I added does not show
q1 = cd.QuestionFactory(questions["Q6-1"]).serve()
q2 = cd.QuestionFactory(questions["Q6-2"]).serve()
q3 = cd.QuestionFactory(questions["Q6-3"]).serve()
q4 = cd.QuestionFactory(questions["Q6-4"]).serve()
q5 = cd.QuestionFactory(questions["Q6-5"]).serve()
q6 = cd.QuestionFactory(questions["Q6-6"]).serve()

pn.Column(q1, q2, q3, q4, q5, q6)

Column
    [0] Column
        [0] StaticText(name='Q6-1', value='What is the d...)
        [1] RadioBoxGroup(name='Options', options=['offshore', 'onshore', ...], value='offshore')
        [2] Button(name='Submit')
        [3] StaticText()
    [1] Column
        [0] StaticText(name='Q6-2', value='What is the d...)
        [1] RadioBoxGroup(name='Options', options=['offshore', 'onshore', ...], value='offshore')
        [2] Button(name='Submit')
        [3] StaticText()
    [2] Column
        [0] StaticText(name='Q6-3', value='What is the d...)
        [1] RadioBoxGroup(name='Options', options=['offshore', 'onshore', ...], value='offshore')
        [2] Button(name='Submit')
        [3] StaticText()
    [3] Column
        [0] StaticText(name='Q6-4', value='What is the d...)
        [1] RadioBoxGroup(name='Options', options=['offshore', 'onshore', ...], value='offshore')
        [2] Button(name='Submit')
        [3] StaticText()
    [4] Column
        [0] StaticText(name='Q6-5', value='For what conditions i...)
        [1] RadioBoxGroup(name='Options', options=['storm conditions', ...], value='storm conditions')
        [2] Button(name='Submit')
        [3] StaticText()
    [5] Column
        [0] StaticText(name='Q6-6', value='What morphological r...)
        [1] CheckBoxGroup(name='Options', options=['erosion higher in the pr...])
        [2] Button(name='Submit')
        [3] StaticText()

## Define shape functions
Below the shape functions by Tinker et al (2009) are defined. They differentiate between transport in the surf/shoaling zone, and the swash/(inner) surf zone. Transport in the surf/shoaling zone was separated in a mean and oscillatory transport (or flux), and transport in the swash/surf zone in an onshore and offshore component. This means that in the surf/shoaling zone, both high- and low-frequency waves are combined in the 'oscillatory' component, which is different than in Figure 7.21 from the book.

You should be somewhat familiar with the concept of mean transport (as a result of undertow) and oscillatory transport (resulting from high- and low-frequency waves) in the shoaling and surf zone. However, the onshore and offshore transport for the swash/inner surf zone are newly introduced here. The transport in the swash zone can be quite important, but it is not easy to model and it is therefore not always taken into account in models. Also, the study of Roelvink and Stive (figure 7.21) did not take the swash zone into account. As expected, the onshore transport component of the swash /inner surf zone is always positive (i.e. onshore), whereas the offshore transport component is always negative. The onshore transport occurs under all conditions and is confined to very shalllow water, whereas the offshore component only occurs during high-energy conditions and takes place a bit furher offshore in the inner surf zone (but still in the swash/inner surf zone). During high-energy conditions, the offshore transport function helps explain rapid erosion in the outer swash/inner surf zone, by pulling the offshore transport maximum shorewards.

Furthermore, these shape functions represent suspended sediment transport only. The measurements by Tinker et al. were done at Sennen Cove, Cornwall, UK. For some context, the mean spring tidal range (MSTR) at this site is 5.3 m, which makes this a macrotidal system. The average significant wave height is 1.4 m, and the median grain size is 0.7 mm. The bed profile is discussed further below.

**Note:** You are not expected to know or remember these equations, they are just here to help us with the exercise.

In [4]:
# Shape functions for the surf/shoal zone

def Tinker_mean(h, h_b):
    return (-120 * h_b**2) * (h / h_b)**4.3 * np.exp(-9.4 * h / h_b**0.75)

def Tinker_osci(h, h_b):
    return (2.75 * h_b**0.6) * (h / h_b)**3.5 * np.exp(-4.2 * h / h_b**1.05)

def Tinker_surf_shoal(h, h_b):
    return Tinker_mean(h, h_b) + Tinker_osci(h, h_b)

# Shape functions for the swash/surf zone

def Tinker_onsh(h, h_b):
    return 3.5 * h_b**1.9 * (h / h_b)**1.1 * np.exp(-31 * (h / h_b)**1.1)**(h_b**1.1)

def Tinker_offs(h, h_b):
    
    if h_b > 2.15:
        a_off = -3 * h_b + 4
    elif h_b <= 0.75:
        a_off = 0
    else:
        a_off = -1.25 * (h_b - 0.75)**2
        
    return a_off * h_b**1.1 * (h / h_b)**1.1 * np.exp(-5.7 * (h/h_b)**1.1)**(h_b**1.1)

def Tinker_swash_surf(h, h_b):
    return Tinker_onsh(h, h_b) + Tinker_offs(h, h_b)

# Total transport for swash/surf/shoal

def Tinker_total(h, h_b):
    return Tinker_surf_shoal(h, h_b) + Tinker_swash_surf(h, h_b)


Let's start with just looking at the surf/shoaling zone. As you can see the shape functions depend on the local water depth $h$ and on the water depth at breaking $h_b$. The larger $h_b$, the more energetic the waves. Remember that in notebook 3c you have determined $h_b$ as part of the calculation of the cross-shore distribution of the wave heights in the shoaling zone and surf zone.

Let's try these functions for different types of bathymetry, assuming some breakpoint depth $h_b$! 


### Define bathymetry functions
The functions below define the bathymetry. Two different bathymetries are defined. The first is used by Tinker et al. (2009) to define their shape functions. As mentioned, this is the bathymetry at Sennen Cove, Cornwall, UK. The second bathymetry is the bathymetry shown in Figure 7.21 in the book. Roelvink and Stive (1989) used this bathymetry for flume experiments performed at Deltares (formerly Delft Hydraulics).
**@Kevin: could we have a third profile here, for instance the Egmond parabolic fit from Figure 7.8? In that way we can compare the UK profile to the Dutch profile. I would call the profiles Sennen Cove, UK, Egmond, Netherlands and Scheldt flume.**

In [5]:
# Bathymetries here!
uk_bath = pd.read_csv("6_uk_bath.csv", sep='; ', decimal=',', names=['x', 'y'], engine='python')
nl_bath = pd.read_csv("6_nl_bath.csv", sep='; ', decimal=',', names=['x', 'y'], engine='python')

def bath(loc):
    if loc == "Sennen Cove, Cornwall (UK)":
        return uk_bath
    elif loc == "Scheldt flume (NL)":
        return nl_bath
    else:
        print("Choose either 'Sennen Cove, Cornwall (UK)' or 'Scheldt flume (NL)'")

### Define plot function

**Note**: Although you don't have to understand the plot method, we include it here so you can see how these interactive plots are made!

In [6]:
def show_transport(plot_where="pop-out"):
    """
    change value of 'plot_where' to:
    'inline' if you would like the plot to show in the notebook
    'pop-out' if you would like the plot to show in a new tab (i.e. seperate window)
    """

    sush_title = pn.pane.Markdown("#### Surf/shoal zone", styles={"color": "black"}, height=25)
    swsu_title = pn.pane.Markdown("#### Swash/surf zone", styles={"color": "black"}, height=25)
    tota_title = pn.pane.Markdown("#### All zones", styles={"color": "black"}, height=25)

    hb_slider = pn.widgets.FloatSlider(name="Breaker depth [m]:", start=0.05, end=3, step=0.05, value=0.8)

    wl_slider = pn.widgets.FloatSlider(name="Water Level [m] (relative to reference):", start=-2, end=2, step=0.05, value=0)

    A_input = pn.widgets.FloatInput(name="A (Parabolic profile):", value=0.043)
    m_input = pn.widgets.FloatInput(name="m (Parabolic profile):", value=0.773)

    bath_dropdown =  pn.widgets.Select(name="Bathymetry select:", options=["Sennen Cove, Cornwall (UK)", "Scheldt flume (NL)", "Parabolic profile (A, m)"], value="Sennen Cove, Cornwall (UK)")

    mean_switch = pn.widgets.Checkbox(name="Mean transport (surf / shoaling zone)", value=True)
    osci_switch = pn.widgets.Checkbox(name="Oscillatory transport (surf / shoaling zone)", value=True)
    sush_switch = pn.widgets.Checkbox(name="Total transport (surf / shoaling zone)", value=False)
    
    onsh_switch = pn.widgets.Checkbox(name="Onshore transport (swash / surf zone)", value=False)
    offs_switch = pn.widgets.Checkbox(name="Offshore transport (swash / surf zone)", value=False)
    swsu_switch = pn.widgets.Checkbox(name="Total transport (swash / surf zone)", value=False)

    tota_switch = pn.widgets.Checkbox(name="Total transport (all)", value=False)

    @pn.depends(wl_slider.param.value, 
                hb_slider.param.value, 
                bath_dropdown.param.value,
                A_input.param.value, 
                m_input.param.value,
                mean_switch.param.value,
                osci_switch.param.value,
                sush_switch.param.value,
                onsh_switch.param.value,
                offs_switch.param.value,
                swsu_switch.param.value,
                tota_switch.param.value
               )
    def plot(wl, h_b, loc, A, m,
             include_mean, include_osci, include_sush, include_onsh, include_offs, include_swsu, include_tota):
        if loc in ["Sennen Cove, Cornwall (UK)", "Scheldt flume (NL)",]:
            df = bath(loc)
            x = df.x
            y = df.y

        elif loc in ["Parabolic profile (A, m)"]:
            x = np.linspace(0, 1000, 500)
            y = -A * x**m
            
        else:
            print("Choose either 'Sennen Cove, Cornwall (UK)', 'Scheldt flume (NL)', or 'Parabolic profile (A, m)'")
        
        h = np.maximum(np.zeros(y.shape), wl - y)

        x_b = x[np.argmin(np.abs(h - h_b))]
        
        
        bath_plot = (hv.Curve((x, y), label='Bathymetry').opts(xlabel='x [m]', ylabel='z [m]', color='black') * \
                    hv.HLine(wl, label='water level').opts(line_dash='dashed') * \
                    hv.VLine(x_b, label='location of breaking').opts(line_dash='dashed', color='grey')).opts(title='Bathymetry')

        curve_transport = hv.Curve(([],[]))
        curve_h_hb = hv.Curve(([],[]))

        if include_mean:
            mean_transport = Tinker_mean(h, h_b)
            curve_transport *= hv.Curve((x, mean_transport), label='mean transport').opts(color='crimson')
            curve_h_hb *= hv.Curve((np.linspace(0, 6, 500)/h_b, Tinker_mean(np.linspace(0, 6, 500), h_b)), label='mean transport').opts(color='crimson')

        if include_osci:
            osci_transport = Tinker_osci(h, h_b)
            curve_transport *= hv.Curve((x, osci_transport), label='oscillatory transport').opts(color='orange')
            curve_h_hb *= hv.Curve((np.linspace(0, 6, 500)/h_b, Tinker_osci(np.linspace(0, 6, 500), h_b)), label='oscillatory transport').opts(color='orange')

        if include_sush:
            sush_transport = Tinker_surf_shoal(h, h_b)
            curve_transport *= hv.Curve((x, sush_transport), label='Total transport (surf/shoal)').opts(color='#aea04b')
            curve_h_hb *= hv.Curve((np.linspace(0, 6, 500)/h_b, Tinker_surf_shoal(np.linspace(0, 6, 500), h_b)), label='Total transport (surf/shoal)').opts(color='#aea04b')
            
        if include_onsh:
            onsh_transport = Tinker_onsh(h, h_b)
            curve_transport *= hv.Curve((x, onsh_transport), label='onshore transport').opts(color='green')
            curve_h_hb *= hv.Curve((np.linspace(0, 6, 500)/h_b, Tinker_onsh(np.linspace(0, 6, 500), h_b)), label='onshore transport').opts(color='green')

        if include_offs:
            offs_transport = Tinker_offs(h, h_b)
            curve_transport *= hv.Curve((x, offs_transport), label='offshore transport').opts(color='blue')
            curve_h_hb *= hv.Curve((np.linspace(0, 6, 500)/h_b, Tinker_offs(np.linspace(0, 6, 500), h_b)), label='offshore transport').opts(color='blue')

        if include_swsu:
            swsu_transport = Tinker_swash_surf(h, h_b)
            curve_transport *= hv.Curve((x, swsu_transport), label='Total transport (swash/surf)').opts(color='#FF00FF')
            curve_h_hb *= hv.Curve((np.linspace(0, 6, 500)/h_b, Tinker_swash_surf(np.linspace(0, 6, 500), h_b)), label='Total transport (swash/surf)').opts(color='#FF00FF')

        if include_tota:
            tota_transport = Tinker_total(h, h_b)
            curve_transport *= hv.Curve((x, tota_transport), label='Total transport (all)').opts(color='grey')
            curve_h_hb *= hv.Curve((np.linspace(0, 6, 500)/h_b, Tinker_total(np.linspace(0, 6, 500), h_b)), label='Total transport (all)').opts(color='grey')
        
        curve_transport *= hv.HLine(0).opts(color='black')
        curve_h_hb *=  hv.HLine(0).opts(color='black')

        transport_plot = curve_transport.opts(xlabel='x [m]', ylabel='Q [kg/m/s]', title='Transport (x)')
        h_hb_plot = curve_h_hb.opts(xlabel='h / h_b [-]', ylabel='Q [kg/m/s]', title='Transport (h/h_b)')
        
        p = (bath_plot.opts(
            height=200, width=1000, show_grid=True, active_tools=[], toolbar=None, legend_position='bottom_left') + \
         transport_plot.opts(
             height=250, width=1000, show_grid=True, active_tools=[], toolbar=None, legend_position='bottom_left').redim(y=hv.Dimension('Transport (x)', soft_range=(-0.1, 0.1))) + \
         h_hb_plot.opts(
             height=200, width=1000, show_grid=True, active_tools=[], toolbar=None, legend_position='bottom_right').redim(x=hv.Dimension('h / h_b [-]', range=(0, 2)), y=hv.Dimension('Transport (h /h_b)', soft_range=(-0.1, 0.1)))
            ).opts(shared_axes=False).cols(1)

        return p

    app = pn.Column(
        pn.Row(bath_dropdown, pn.Column(wl_slider, hb_slider), pn.Column(A_input, m_input), align='center'),
        pn.Row(pn.Column(sush_title, mean_switch, osci_switch, sush_switch), 
               pn.Column(swsu_title, onsh_switch, offs_switch, swsu_switch),
               pn.Column(tota_title, tota_switch), align='center'), 
        pn.Row(plot, align='center')
    )
    
    if plot_where == "inline":
        return app
    elif plot_where == "pop-out":
        app.show()
    else:
        print("please use either inline or pop-out for the plot_where variable")

### Now plot the results

Execute the cell below to generate the plot by using the function we defined above. Please note that altering the slider positions or selecting different options from the dropdown menus may trigger a warning; it can safely be ignored, and possibly silenced by the adjusting the logging warning level. 

In [7]:
warnings.filterwarnings("ignore")
logging.getLogger().setLevel(logging.ERROR)

show_transport(plot_where="pop-out")

Launching server at http://localhost:59503


You can select the bathymetry you want to look at using the selector in the panel. Use the sliders to change the water level and breaker depth (plotted as dashed blue and gray line respectively). Finally, you can choose to whether include certain transport components in the transport plot or not. By default, only the mean transport and the oscillatory transport are included. For now, let's keep it that way. Using this plot, try to answer the questions below.

In [8]:
# The unit of transport used in the plot is kg/m/s. Question about this?
q7 = cd.QuestionFactory(questions["Q6-7"]).serve()

pn.Column(q7)

Column
    [0] Column
        [0] StaticText(name='Q6-7', value='The unit of t...)
        [1] RadioBoxGroup(name='Options', options=['We are considering sedim...], value='We are considering s...)
        [2] Button(name='Submit')
        [3] StaticText()

### Sennen cove, Cornwall
We will start by looking at the United Kingdom site where the measurements were done. The measurements were done over many tides (39) and for a range of wave conditions (offshore significant waveheights ranging from 0.1–2.5m. Firstly, we will look at the more energetic conditions, which might occur during a storm. According to Tinker et al., a breaker depth of $h_b=2$ m is representative of such a situation. Set the water level to MSL +2 m (this is MHWS). 

**@Kevin: what is this choice of MSL + 2 m based on?**

**@Kevin: for these conditions the vertical scale in the lower plot does not capture the mean transport entirely**

In [9]:
# @Kevin, Is it possible to convert Q6-9 to a numeric question? 
q8 = cd.QuestionFactory(questions["Q6-8"]).serve()
q9 = cd.QuestionFactory(questions["Q6-9"]).serve()
q10 = cd.QuestionFactory(questions["Q6-10"]).serve()
q11 = cd.QuestionFactory(questions["Q6-11"]).serve()
q12 = cd.QuestionFactory(questions["Q6-12"]).serve()
q13 = cd.QuestionFactory(questions["Q6-13"]).serve()
q14 = cd.QuestionFactory(questions["Q6-14"]).serve()

pn.Column(q8, q9, q10, q11, q12, q13, q14)

Column
    [0] Column
        [0] StaticText(name='Q6-8', value='What morphological r...)
        [1] CheckBoxGroup(name='Options', options=['erosion higher in the pr...])
        [2] Button(name='Submit')
        [3] StaticText()
    [1] Column
        [0] StaticText(name='Q6-9', value='How wide is the surf zone...)
        [1] FloatInput(name='Your Answer')
        [2] Button(name='Submit')
        [3] StaticText()
    [2] Column
        [0] StaticText(name='Q6-10', value='What is the d...)
        [1] RadioBoxGroup(name='Options', options=['onshore, onshore', ...], value='onshore, onshore')
        [2] Button(name='Submit')
        [3] StaticText()
    [3] Column
        [0] StaticText(name='Q6-11', value='Where do we s...)
        [1] RadioBoxGroup(name='Options', options=['In the shoaling zone and...], value='In the shoaling z...)
        [2] Button(name='Submit')
        [3] StaticText()
    [4] Column
        [0] StaticText(name='Q6-12', value='Where do we s...)
        [1] RadioBoxGroup(name='Options', options=['In the shoaling zone and...], value='In the shoaling z...)
        [2] Button(name='Submit')
        [3] StaticText()
    [5] Column
        [0] StaticText(name='Q6-13', value='What is the d...)
        [1] RadioBoxGroup(name='Options', options=['Onshore for h/h_b > 1, ...], value='Onshore for h/h_b > 1, ...)
        [2] Button(name='Submit')
        [3] StaticText()
    [6] Column
        [0] StaticText(name='Q6-14', value='Compare this r...)
        [1] CheckBoxGroup(name='Options', options=['...', '...'])
        [2] Button(name='Submit')
        [3] StaticText()

**@Judith Add Summary: see end page 5, start page 6. Make comment about Q6-14 and that data did not extend into surf zone of high energy waves.**
**Also add Scheldt flume => compare with velocity moments, again: shoaling zone uncertain**
Now we continue with looking at low-energy conditions, which according to Tinker et al. is represented by a breaking depth $h_b=0.6$ m. Change the value of $h_b$ in the plot to 0.6 m, and try to answer the questions below.
**And what about the MSL?**

In [10]:
# Do you expect accretion or erosion for the situation described above?
# How wide is the surf zone approximately?
# What is the direction of transport due to waves? And due to undertow?
# Where do we expect onshore transport (h/h_b)?
# Where do we expect offshore transport (h/h_b)?
# What is the net direction of the total transport (h/h_b)?

q15 = cd.QuestionFactory(questions["Q6-15"]).serve()
q16 = cd.QuestionFactory(questions["Q6-16"]).serve()
q17 = cd.QuestionFactory(questions["Q6-17"]).serve()
q18 = cd.QuestionFactory(questions["Q6-18"]).serve()
q19 = cd.QuestionFactory(questions["Q6-19"]).serve()
q20 = cd.QuestionFactory(questions["Q6-20"]).serve()

pn.Column(q15, q16, q17, q18, q19, q20)

Column
    [0] Column
        [0] StaticText(name='Q6-15', value='Do you expect a...)
        [1] RadioBoxGroup(name='Options', options=['accretion', 'erosion'], value='accretion')
        [2] Button(name='Submit')
        [3] StaticText()
    [1] Column
        [0] StaticText(name='Q6-16', value='How wide is the surf zone...)
        [1] FloatInput(name='Your Answer')
        [2] Button(name='Submit')
        [3] StaticText()
    [2] Column
        [0] StaticText(name='Q6-17', value='What is the d...)
        [1] RadioBoxGroup(name='Options', options=['oshore, onshore', ...], value='oshore, onshore')
        [2] Button(name='Submit')
        [3] StaticText()
    [3] Column
        [0] StaticText(name='Q6-18', value='Where do we e...)
        [1] RadioBoxGroup(name='Options', options=['...', '...'], value='...')
        [2] Button(name='Submit')
        [3] StaticText()
    [4] Column
        [0] StaticText(name='Q6-19', value='Where do we e...)
        [1] RadioBoxGroup(name='Options', options=['...', '...'], value='...')
        [2] Button(name='Submit')
        [3] StaticText()
    [5] Column
        [0] StaticText(name='Q6-20', value='What is the n...)
        [1] RadioBoxGroup(name='Options', options=['...', '...'], value='...')
        [2] Button(name='Submit')
        [3] StaticText()

Now we will reflect on the differences between high- and low-energy conditions. Answer the questions below.

In [11]:
# Some reflective questions
q21 = cd.QuestionFactory(questions["Q6-21"]).serve()
q22 = cd.QuestionFactory(questions["Q6-22"]).serve()
q23 = cd.QuestionFactory(questions["Q6-23"]).serve()

pn.Column(q21, q22, q23)

Column
    [0] Column
        [0] StaticText(name='Q6-21', value='Reflective question 1')
        [1] RadioBoxGroup(name='Options', options=['...', '...'], value='...')
        [2] Button(name='Submit')
        [3] StaticText()
    [1] Column
        [0] StaticText(name='Q6-22', value='Reflective question 2')
        [1] RadioBoxGroup(name='Options', options=['...', '...'], value='...')
        [2] Button(name='Submit')
        [3] StaticText()
    [2] Column
        [0] StaticText(name='Q6-23', value='Reflective question 3')
        [1] RadioBoxGroup(name='Options', options=['...', '...'], value='...')
        [2] Button(name='Submit')
        [3] StaticText()

Above we have seen that a key feature of the two shape functions is that both the mean and oscillatory sediment fluxes increase with hb, but that the offshore-directed mean flux increases faster than the onshore-directed oscillatory flux. Consequently, the net transport changes from onshore to offshore around $h_b$ = 1.7m (representing an offshore significant wave height of around Hsig  = 0.8 m). **Can make a question about this hb= 1.7m**


In [12]:
### For question mentioned above

### Scheldt flume
The shape functions from Tinker et al. were developed for the UK site. Let's check how well they perform for a typical Dutch profile, for example the one used by Roelvink and Stive for their flume experiments. You can switch between the two bathymetries using the dropdown menu with the plot.
**Scheldt flume and typical Dutch profile is something else** 

In [13]:
# For what range of values for h_b do you expect erosive conditions for the Dutch profile?
# For what range of values for h_b do you expect accretive conditions for the Dutch profile?
# What differences do you observe between the UK and the Dutch profile?
# What similarities do you observe between the UK and the Dutch profile?

q24 = cd.QuestionFactory(questions["Q6-24"]).serve()
q25 = cd.QuestionFactory(questions["Q6-25"]).serve()
q26 = cd.QuestionFactory(questions["Q6-26"]).serve()
q27 = cd.QuestionFactory(questions["Q6-27"]).serve()

pn.Column(q24, q25, q26, q27)

Column
    [0] Column
        [0] StaticText(name='Q6-24', value='For what range o...)
        [1] RadioBoxGroup(name='Options', options=['...', '...'], value='...')
        [2] Button(name='Submit')
        [3] StaticText()
    [1] Column
        [0] StaticText(name='Q6-25', value='For what range o...)
        [1] RadioBoxGroup(name='Options', options=['...', '...'], value='...')
        [2] Button(name='Submit')
        [3] StaticText()
    [2] Column
        [0] StaticText(name='Q6-26', value='What differences d...)
        [1] CheckBoxGroup(name='Options', options=['...', '...'])
        [2] Button(name='Submit')
        [3] StaticText()
    [3] Column
        [0] StaticText(name='Q6-27', value='What similarities d...)
        [1] RadioBoxGroup(name='Options', options=['...', '...'], value='...')
        [2] Button(name='Submit')
        [3] StaticText()

Including the on- and offshore transport shape functions allows us to look at what happens in the swash and the inner surf zone. Enable these functions in the plot, and try to answer the questions below.

In [14]:
# Some reflective questions
q28 = cd.QuestionFactory(questions["Q6-28"]).serve()
q29 = cd.QuestionFactory(questions["Q6-29"]).serve()
q30 = cd.QuestionFactory(questions["Q6-30"]).serve()

pn.Column(q28, q29, q30)

Column
    [0] Column
        [0] StaticText(name='Q6-28', value='Reflective question 1')
        [1] RadioBoxGroup(name='Options', options=['...', '...'], value='...')
        [2] Button(name='Submit')
        [3] StaticText()
    [1] Column
        [0] StaticText(name='Q6-29', value='Reflective question 2')
        [1] RadioBoxGroup(name='Options', options=['...', '...'], value='...')
        [2] Button(name='Submit')
        [3] StaticText()
    [2] Column
        [0] StaticText(name='Q6-30', value='Reflective question 3')
        [1] RadioBoxGroup(name='Options', options=['...', '...'], value='...')
        [2] Button(name='Submit')
        [3] StaticText()

### ($h / h_b$)-plot
As you will have noticed by now, the plotting function generates three subplots: the bathymetry, the transport as a function of x, and the transport as a function of $h/h_b$. The dimensionless $h/h_b$ is a convenient way to normalize cross-shore profiles, to see where accretion/erosion might occur. Below, a new plotting method is defined to look in this manner of plotting more closely.

**Note**: Although you don't have to understand the plot method, we include it here so you can see how these interactive plots are made!

In [15]:
def h_hb_transport(plot_where="pop-out"):
    """
    change value of 'plot_where' to:
    'inline' if you would like the plot to show in the notebook
    'pop-out' if you would like the plot to show in a new tab (i.e. seperate window)
    """
    sush_title = pn.pane.Markdown("#### Surf/shoal zone", styles={"color": "black"}, height=25)
    swsu_title = pn.pane.Markdown("#### Swash/surf zone", styles={"color": "black"}, height=25)
    tota_title = pn.pane.Markdown("#### All zones", styles={"color": "black"}, height=25)
    
    hb_slider = pn.widgets.RangeSlider(name="Range of hb [m] values to plot", start=0, end=2.5, value=(0, 2.5))

    mean_switch = pn.widgets.Checkbox(name="Mean transport (surf / shoaling zone)", value=True)
    osci_switch = pn.widgets.Checkbox(name="Oscillatory transport (surf / shoaling zone)", value=True)
    sush_switch = pn.widgets.Checkbox(name="Total transport (surf / shoaling zone)", value=False)
    
    onsh_switch = pn.widgets.Checkbox(name="Onshore transport (swash / surf zone)", value=False)
    offs_switch = pn.widgets.Checkbox(name="Offshore transport (swash / surf zone)", value=False)
    swsu_switch = pn.widgets.Checkbox(name="Total transport (swash / surf zone)", value=False)

    tota_switch = pn.widgets.Checkbox(name="Total transport (all)")

    @pn.depends(hb_slider.param.value, 
                mean_switch.param.value,
                osci_switch.param.value,
                sush_switch.param.value,
                onsh_switch.param.value,
                offs_switch.param.value,
                swsu_switch.param.value,
                tota_switch.param.value
               )
    def plot(h_b,
             include_mean, include_osci, include_sush, include_onsh, include_offs, include_swsu, include_tota):

        hb1 = h_b[0]
        hb4 = h_b[1]
        hb2 = (hb4-hb1) * 1/3 + hb1
        hb3 = (hb4-hb1) * 2/3 + hb1

        curve_transport1 = hv.Curve(([],[]))
        curve_transport2 = hv.Curve(([],[]))
        curve_transport3 = hv.Curve(([],[]))
        curve_transport4 = hv.Curve(([],[]))

        h = np.linspace(0, 5, 500)

        if include_mean:
            curve_transport1 *= hv.Curve((h/hb1, Tinker_mean(h, hb1)), label='mean transport').opts(color='crimson')
            curve_transport2 *= hv.Curve((h/hb2, Tinker_mean(h, hb2)), label='mean transport').opts(color='crimson')
            curve_transport3 *= hv.Curve((h/hb3, Tinker_mean(h, hb3)), label='mean transport').opts(color='crimson')
            curve_transport4 *= hv.Curve((h/hb4, Tinker_mean(h, hb4)), label='mean transport').opts(color='crimson')

        if include_osci:
            curve_transport1 *= hv.Curve((h/hb1, Tinker_osci(h, hb1)), label='oscillatory transport').opts(color='orange')
            curve_transport2 *= hv.Curve((h/hb2, Tinker_osci(h, hb2)), label='oscillatory transport').opts(color='orange')
            curve_transport3 *= hv.Curve((h/hb3, Tinker_osci(h, hb3)), label='oscillatory transport').opts(color='orange')
            curve_transport4 *= hv.Curve((h/hb4, Tinker_osci(h, hb4)), label='oscillatory transport').opts(color='orange')

        if include_sush:
            curve_transport1 *= hv.Curve((h/hb1, Tinker_surf_shoal(h, hb1)), label='Total transport (surf/shoal)').opts(color='#aea04b')
            curve_transport2 *= hv.Curve((h/hb2, Tinker_surf_shoal(h, hb2)), label='Total transport (surf/shoal)').opts(color='#aea04b')
            curve_transport3 *= hv.Curve((h/hb3, Tinker_surf_shoal(h, hb3)), label='Total transport (surf/shoal)').opts(color='#aea04b')
            curve_transport4 *= hv.Curve((h/hb4, Tinker_surf_shoal(h, hb4)), label='Total transport (surf/shoal)').opts(color='#aea04b')

        if include_onsh:
            curve_transport1 *= hv.Curve((h/hb1, Tinker_onsh(h, hb1)), label='onshore transport').opts(color='green')
            curve_transport2 *= hv.Curve((h/hb2, Tinker_onsh(h, hb2)), label='onshore transport').opts(color='green')
            curve_transport3 *= hv.Curve((h/hb3, Tinker_onsh(h, hb3)), label='onshore transport').opts(color='green')
            curve_transport4 *= hv.Curve((h/hb4, Tinker_onsh(h, hb4)), label='onshore transport').opts(color='green')

        if include_offs:
            curve_transport1 *= hv.Curve((h/hb1, Tinker_offs(h, hb1)), label='offshore transport').opts(color='blue')
            curve_transport2 *= hv.Curve((h/hb2, Tinker_offs(h, hb2)), label='offshore transport').opts(color='blue')
            curve_transport3 *= hv.Curve((h/hb3, Tinker_offs(h, hb3)), label='offshore transport').opts(color='blue')
            curve_transport4 *= hv.Curve((h/hb4, Tinker_offs(h, hb4)), label='offshore transport').opts(color='blue')

        if include_swsu:
            curve_transport1 *= hv.Curve((h/hb1, Tinker_swash_surf(h, hb1)), label='Total transport (swash/surf)').opts(color='#FF00FF')
            curve_transport2 *= hv.Curve((h/hb2, Tinker_swash_surf(h, hb2)), label='Total transport (swash/surf)').opts(color='#FF00FF')
            curve_transport3 *= hv.Curve((h/hb3, Tinker_swash_surf(h, hb3)), label='Total transport (swash/surf)').opts(color='#FF00FF')
            curve_transport4 *= hv.Curve((h/hb4, Tinker_swash_surf(h, hb4)), label='Total transport (swash/surf)').opts(color='#FF00FF')

        if include_tota:
            curve_transport1 *= hv.Curve((h/hb1, Tinker_total(h, hb1)), label='Total transport (all)').opts(color='grey')
            curve_transport2 *= hv.Curve((h/hb2, Tinker_total(h, hb2)), label='Total transport (all)').opts(color='grey')
            curve_transport3 *= hv.Curve((h/hb3, Tinker_total(h, hb3)), label='Total transport (all)').opts(color='grey')
            curve_transport4 *= hv.Curve((h/hb4, Tinker_total(h, hb4)), label='Total transport (all)').opts(color='grey')
        
        curve_transport1 *= hv.HLine(0).opts(color='black')
        curve_transport2 *= hv.HLine(0).opts(color='black')
        curve_transport3 *= hv.HLine(0).opts(color='black')
        curve_transport4 *= hv.HLine(0).opts(color='black')

        transport_plot1 = curve_transport1.opts(xlabel='h / h_b [-]', ylabel='Q [kg/m/s]', title=f'Transport (hb={hb1:.1f})')
        transport_plot2 = curve_transport2.opts(xlabel='h / h_b [-]', ylabel='Q [kg/m/s]', title=f'Transport (hb={hb2:.1f})')
        transport_plot3 = curve_transport3.opts(xlabel='h / h_b [-]', ylabel='Q [kg/m/s]', title=f'Transport (hb={hb3:.1f})')
        transport_plot4 = curve_transport4.opts(xlabel='h / h_b [-]', ylabel='Q [kg/m/s]', title=f'Transport (hb={hb4:.1f})')

        p = (transport_plot4.opts(
                            height=175, width=1000, show_grid=True, active_tools=[], toolbar=None, legend_position='bottom_right').redim(
                            x=hv.Dimension('h / h_b [-]', range=(0, 2)), y=hv.Dimension('Transport (h /h_b)', soft_range=(-0.1, 0.1))) + \
            transport_plot3.opts(
                            height=175, width=1000, show_grid=True, active_tools=[], toolbar=None, legend_position='bottom_right').redim(
                            x=hv.Dimension('h / h_b [-]', range=(0, 2)), y=hv.Dimension('Transport (h /h_b)', soft_range=(-0.1, 0.1))) + \
            transport_plot2.opts(
                            height=175, width=1000, show_grid=True, active_tools=[], toolbar=None, legend_position='bottom_right').redim(
                            x=hv.Dimension('h / h_b [-]', range=(0, 2)), y=hv.Dimension('Transport (h /h_b)', soft_range=(-0.1, 0.1))) + \
            transport_plot1.opts(
                            height=175, width=1000, show_grid=True, active_tools=[], toolbar=None, legend_position='bottom_right').redim(
                            x=hv.Dimension('h / h_b [-]', range=(0, 2)), y=hv.Dimension('Transport (h /h_b)', soft_range=(-0.1, 0.1)))
            ).cols(1)

        return p

    app = pn.Column(
        pn.Row(hb_slider, align='center'),
        pn.Row(pn.Column(sush_title, mean_switch, osci_switch, sush_switch), 
               pn.Column(swsu_title, onsh_switch, offs_switch, swsu_switch),
               pn.Column(tota_title, tota_switch), align='center'), 
        pn.Row(plot, align='center')
    )
    
    if plot_where == "inline":
        return app
    elif plot_where == "pop-out":
        app.show()
    else:
        print("please use either inline or pop-out for the plot_where variable")

Execute the cell below to generate the plot by using the function we defined above. Please note that altering the slider positions or selecting different options from the dropdown menus may trigger a warning; it can safely be ignored, and possibly silenced by the adjusting the logging warning level. 

In [16]:
warnings.filterwarnings("ignore")
logging.getLogger().setLevel(logging.ERROR)

h_hb_transport(plot_where="pop-out")

Launching server at http://localhost:59505


Try to answer the questions below using this plot.

In [17]:
# Some reflective questions
q31 = cd.QuestionFactory(questions["Q6-31"]).serve()
q32 = cd.QuestionFactory(questions["Q6-32"]).serve()
q33 = cd.QuestionFactory(questions["Q6-33"]).serve()

pn.Column(q31, q32, q33)

Column
    [0] Column
        [0] StaticText(name='Q6-31', value='Reflective question 1')
        [1] RadioBoxGroup(name='Options', options=['...', '...'], value='...')
        [2] Button(name='Submit')
        [3] StaticText()
    [1] Column
        [0] StaticText(name='Q6-32', value='Reflective question 2')
        [1] RadioBoxGroup(name='Options', options=['...', '...'], value='...')
        [2] Button(name='Submit')
        [3] StaticText()
    [2] Column
        [0] StaticText(name='Q6-33', value='Reflective question 3')
        [1] RadioBoxGroup(name='Options', options=['...', '...'], value='...')
        [2] Button(name='Submit')
        [3] StaticText()

You have reached the end of this notebook. You now know about the different components in the swash, surf, and shoaling zone, and have seen some practical exapmles!

### TO BE REMOVED

**I did not get a selector menu, I also did not get a plot**
**Therefore I cannot really look at the notebook in depth. But here are some ideas for the story line of the remainder of the notebook:**


* First we have to explain some of the assumptions behind the Tinker paper:  we are now using waves (short + long combined) and mean current as opposed to figure 7.21,
* Explain the approach to the swash zone (which extends beyond the focus of 7.21)

* * Ask students to first select the profile for the measurement location and a hb to represent erosive conditions with net offshore transport. Ask questions about:
  -range of hb values for which erosive conditions,
    -for a particular hb in this range:
                                          width of surf zone (in m),
                                      direction of transport due waves and due to undertow,
                                      zones in which onshore and offshore transport are significant (as a function of cross-shore distance and/or h/hb)
                                      and direction of net transport (also as a function of cross-shore distance and/or h/hb)


* Let them compare the results to Figure 7.21.


* Now let them determine the hb range for accretive conditions. Choose a certain hb and answer again the questions about
                                     width of surf zone (in m),
                                      direction of transport due waves and due to undertow,
                                      zones in which onshore and offshore transport are significant (as a function of cross-shore distance and/or h/hb)
                                      and direction of net transport (also as a function of cross-shore distance and/or h/hb)
  
* Now ask reflective questions about the difference between the accretive conditions and erosive conditions
* Now discuss validity for different situations and let them choose a typical Dutch profile. Now let them again find the range of hb for which there are typical erosive and accretive conditions.
* Let them compare the UK situation to the Dutch situation
* Now introduce the way of plotting in terms of h/hb in the paper and show these plots. Maybe somne questions to interpret this.
* To be determined: which role do we have for the various bathymetries




TODOS:
* FIX AXIS TOGETHER
* MAKE FIGURE NON-INTERACTABLE
* MAKE FIGURES MOVE TOGETHER
* PLOT OTHER TRANSPORT COMPONENTS
* ADD TINKER PLOT WHERE EVERYTHING IS STACKED TOGETHER
* PRESENT IN TWO FIGURES TO HELP STUDENTS COMPARE HIGH WAVES VS LOW WAVES

* MAKE QUESTIONS (idea: focus on when these shape functions might be valid or not, i.e. different situations)
* ADD MARKDOWNS


 Tides with hb between 0.5 and 0.85m were categorised as being
 low energy, while those with hb between 1.8 and 2.1m were
 classed as high energy

 - unit of Q
- interpretation of different scenarios

 $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$